In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as scipy_img
import scipy.interpolate as sci
from cv2 import VideoWriter, VideoWriter_fourcc

In [ ]:
width = 640
height = 480
fps = 15
seconds = 15
frames = int(fps * seconds)

# Could use "blobs.mp4" and the "H264" fourcc codec below.
FILENAME = "blobs.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(fps), (width, height))

blob_w, blob_h = width // 10, height // 10
TOP_SPEED = 4.5

# We'll have some blocks of various sizes make up blobs.
class Blob:
    def __init__(self, x: float, y: float, size: float):
        self.x = x
        self.y = y
        self.size = size
        # pick a random direction
        self.x_delta = (np.random.random() - 0.5) * TOP_SPEED
        self.y_delta = (np.random.random() - 0.5) * TOP_SPEED

    def move(self):
        self.x += self.x_delta
        self.y += self.y_delta

        if self.x < -self.size:
            self.x = blob_w + self.size
        elif self.x > blob_w + self.size:
            self.x = -self.size
            
        if self.y < -self.size:
            self.y = blob_h + self.size
        elif self.y > blob_h + self.size:
            self.y = -self.size

    def draw(self, img):
        cv2.circle(
            img,
            (int(round(self.x)), int(round(self.y))),
            self.size,
            (255, 255, 255),
            thickness=-1,
            lineType=cv2.LINE_AA,
        )


blobs = [
    Blob(
        np.random.randint(blob_w),
        np.random.randint(blob_h),
        np.random.randint(blob_w / 3),
    )
    for _ in range(10)
]

for frame_num in range(frames):
    # Start with a black background.
    frame = np.zeros((height, width, 3), dtype=np.uint8)

    # Redraw blobs each time.
    blob_img = np.zeros((blob_h, blob_w, 3))
    for blob in blobs:
        blob.draw(blob_img)

    x = np.linspace(0, 1, blob_w, endpoint=False)
    y = np.linspace(0, 1, blob_h, endpoint=False)

    to_int = blob_img[:, :, 0]
    kernel = blob_w // 2
    if kernel % 1 == 0:
        kernel += 1
    to_int = cv2.GaussianBlur(to_int, (kernel, kernel), 0.0)
    f = sci.interp2d(x, y, to_int, kind="cubic")
    
    znew = f(np.linspace(0, 1, width), np.linspace(0, 1, height))
    # normalize
    znew -= np.min(znew)
    znew /= np.max(znew)
    znew = (znew * 255).astype(int)
    frame[:, :, 0] = znew

    # Move blocks around
    for blob in blobs:
        blob.move()
    video.write(frame)

video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)